<a href="https://colab.research.google.com/github/Taylor2718/CodingTutorials/blob/BB84-Protocol/BB84_Protocol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random as ran

#All the functions needed in the demonstration.

def Polarization(Base_string, Bit_string, Base_string_2):
  if len(Base_string) == len(Bit_string) == len(Base_string_2):
    a = ''
    for i in range(0, len(Base_string)):
      if Base_string_2[i] == Base_string[i]:
        a += Bit_string[i] 
      else:
        a += str(ran.randint(0,1))
    return a 
  else:
    raise Exception('Number of bases does not match the number of bits')

def Confirm(Base_string, Base_string_2 ):
  Output = []
  if len(Base_string) == len(Base_string_2):
    for i in range(0, len(Base_string_2)):
      if Base_string[i] == Base_string_2[i]:
        Output.append(True)
      else:
        Output.append(False)
    return Output
  else:
    raise Exception('Number of bases used by the reciever does not match the number of bases used by the sender')


def Get_Key(Bit_string, List, Bit_string_2):
  output = ''
  output_2 = ''
  if len(Bit_string) == len(Bit_string_2):
    for i in range(0, len(List)):
      if List[i]:
        output += Bit_string[i]
        output_2 += Bit_string_2[i]
    return output, output_2

def Sharing_of_bits(Shifted_key, Shifted_key_2):
  if len(Shifted_key) == len(Shifted_key_2):
    a = 0 
    b = []
    for i in range(0, len(Shifted_key)):
      if Shifted_key[i] == Shifted_key_2[i]:
        a += 0 
      else:
        a += 1
        b.append(i+1)
    if a == 0:
      print('No Evesdropper is detected')
    elif a == 1:
      print('Evesdropper is detected!!! Bit number %d is different' %b[0])
    else:
      for c in range(0, len(b)):
        print('Evesdropper is detected!!! Bit number %d are different' %b[c])





#**Background**:

The BB84 protocol is a way of secretly distrbuting keys which can be used to encode messages. It relies on the properties of quantum physics. Mainly the Heisenberg Uncertainty principle and the No cloning principle.

### **Heisenberg Uncertainty Principle**:
This principle states that in a quantum system, only one property of a pair of conjugate properties like position and momentum can be known with certainty. This is used as the polarisation of photons on different bases forms a conjugate pair. 

### **No cloning theorem**: 
It is impossible to create an identical copy of an unknown quantum state. This is what allows the precense of the evesdropper to be detected. 

### How It works:

1. The first step is for the sender to generate a string of bits and a string of bases both of length *n*. This corresponds to the sender generating *n* photons and polarizing them in accordance to the chosen bases. 

In [ ]:

Bit_string = '0110101' 
Sender_Bases = 'RDDRDRR' 




2. The next step is to transmit the polarised photon through a quantum channel (this could an optical fibre for example) to the receiver.

3. Once the receiver has got the photons, they will polarise them using their own chosen order of bases. This will give the receiver a corresponding bit string.


In [ ]:
Receiver_Bases = 'RDDDRDR'
Output_string = Polarization(Sender_Bases, Bit_string, Receiver_Bases)
print('The Bit string the Receiver obtains is ' + Output_string)

The Bit string the Receiver obtains is 0111011


4. The receiver then will share there sequence of bases that they used, via a public channel. The sender will then confirm which of the bases are the same that they used. 

In [ ]:
List_of_correctness = Confirm(Sender_Bases, Receiver_Bases) # True is returned if the Bases match and False is returned if the bases are not
print(List_of_correctness)

[True, True, True, False, False, False, True]


5. The new string is created using only the matching bases. This is called the shifted key and is becomes the key used to encode messages. 

In [ ]:
Shifted_Key, Shifted_Key_2 = Get_Key(Output_string, List_of_correctness, Bit_string )
print('The Shifted Key the Receiver obtains is ' + Shifted_Key)
print('The Shifted Key the Sender obtains is ' + Shifted_Key_2)

The Shifted Key the Receiver obtains is 0111
The Shifted Key the Sender obtains is 0111


### Detecting Evesdroppers:
 Lets analyse what happens when an evesdropper is present in the quantum channel. 
 We will set a new bit string and sequence of bases for the the sender to use. 
 The evesdropper will also need to randomly pick a sequence of bases to polarize the photons. 

In [ ]:
Bit_string_2 = '0110101' 
Sender_Bases_2 = 'RDDRDRR'

Evesdropper_Bases = 'DRDRRDR'



The Evesdropper will observe the photons, and polarising the photons and altering the bit string. 

In [ ]:
Altered_Bit_String = Polarization(Sender_Bases_2, Bit_string_2, Evesdropper_Bases)
print(Altered_Bit_String)

1110101


By observing the the photons, the evesdropper changes the photons that the receiver will obtain. 

In [ ]:
Receiver_Bases_2 = 'RDDDRDR'
Received_Bit_String = Polarization(Sender_Bases_2, Bit_string_2, Evesdropper_Bases)
print(Received_Bit_String)


0010011


As the photons the receiver gets are altered, there is now a chance that the receiver gets a different bit to the sender even though the same bases are used by both. Therefore they could have different Shifted keys. 

In [ ]:
Correct_Bases = Confirm(Sender_Bases_2, Receiver_Bases_2)
print(Correct_Bases)

[True, True, True, False, False, False, True]


In [ ]:
New_Shifted_Key, New_Shifted_Key_2 = Get_Key(Received_Bit_String, Correct_Bases, Bit_string_2)
print('The Shifted Key the Receiver obtains is ' + New_Shifted_Key)
print('The Shifted Key the Sender obtains is ' + New_Shifted_Key_2)

The Shifted Key the Receiver obtains is 0011
The Shifted Key the Sender obtains is 0111


To confirm whether the bit strings are the same. The Sender and the receiver will share some of the bits in the shifted key, via a public channel. If any of the bits are different they know an evedropper is present. 

In [ ]:
Sharing_of_bits(New_Shifted_Key, New_Shifted_Key_2) 

Evesdropper is detected!!! Bit number 2 is different


The Function above checks all of the digits but in practice The shifted keys are much longer and only a fraction of those bits are shared.

However there is also a probability that the evesdropper goes undetected. This probability is called the Escape Probability.

To calculate the Escape Probability, we first find the probability of escape if only 1 bit was shared. 

Since there are only 2 possible choices of Bases, the probability that the evesdropper chooses the same base as the sender and the receiver is 50%.

If the choice of base is the same, the evesdropper goes undetected. However if the choice of base is different there is a 50% probability that the bit will be different. Therefore there is a 25% chance that the bit is correct and a 25% chance that the bit is incorrect. 

So the probability that the bit is correct is 75% or 0.75.

This may seem quite high. however if *n* bits are shared the probability that all *n* bits are correct is 0.75 x 0.75 x 0.75 ... = (0.75)^*n*

The makes it extremely less likely the evesdropper escapes undetected. For example if the number of bits shared is 10, the escape probability is approximately 6%. 